In [3]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
import mplfinance as mpf
from mplfinance import plot as mplot
import os
from tensorflow import keras
from matplotlib.image import imread
from datetime import date
import datetime
from datetime import datetime as dt
import matplotlib 
matplotlib.use('agg')
import matplotlib.pyplot as plt
import re
import tensorflow
import cv2 as cv
from tqdm import tqdm
from PIL import ImageFont, ImageDraw, Image
import gc
from datetime import datetime
import pytz
import warnings
warnings.filterwarnings('ignore')

In [4]:
def get_position_sell(symbol, lot, sl , tp):
    symbol_info = mt5.symbol_info(symbol)
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).bid
    deviation = 20
    print(price + (sl * point))
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": price + (sl * price),
        "tp": price - (tp * price),
        "deviation": deviation,
        "comment": "python script open",
        "type_filling":mt5.SYMBOL_TRADE_EXECUTION_INSTANT,
    }
    result = mt5.order_send(request)
    return result

In [5]:
def get_position_buy(symbol, lot, sl , tp):
    symbol_info = mt5.symbol_info(symbol)
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).bid
    deviation = 20
    print(price + (sl * point))
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY,
        "price": price,
        "sl": price - (sl * price),
        "tp": price + (tp * price),
        "deviation": deviation,
        "comment": "python script open",
        "type_filling":mt5.SYMBOL_TRADE_EXECUTION_INSTANT,
    }
    result = mt5.order_send(request)
    return result

In [6]:
currency_pairs= ['AUDCAD', 'AUDCHF','AUDJPY','AUDNZD','AUDUSD','CADCHF','CADJPY','CHFJPY',
                 'EURAUD','EURCAD','EURCHF','EURGBP','EURJPY','EURNZD','EURUSD',
                 'GBPAUD','GBPCAD','GBPCHF','GBPJPY','GBPNZD','GBPUSD','NZDJPY',
                 'NZDCAD','NZDCHF','NZDUSD','USDCAD','USDCHF','USDJPY']

In [7]:
# connect to MetaTrader 5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

mt5.account_info()

AccountInfo(login=2103662, trade_mode=2, leverage=200, limit_orders=0, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=284.63, credit=0.0, profit=0.0, equity=284.63, margin=0.0, margin_free=284.63, margin_level=0.0, margin_so_call=70.0, margin_so_so=40.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Esmaeil Zahedi', server='AMarkets-Real', currency='USD', company='AMarkets')

In [8]:
image_path_1h = r'C:\Users\Administrator\Desktop\Trade_1005\Images_1h'
image_path_4h = r'C:\Users\Administrator\Desktop\Trade_1005\Images_4h'
RecolPath_1h = r'C:\Users\Administrator\Desktop\Trade_1005\Images_1h\Recolored'
RecolPath_4h = r'C:\Users\Administrator\Desktop\Trade_1005\Images_4h\Recolored'
model_1h= tensorflow.keras.models.load_model('ckpt_03_CorrectHighLowWithoutResize.hdf5')
model_4h_old= tensorflow.keras.models.load_model('ckpt_04_4h_best.hdf5')
model_4h_new= tensorflow.keras.models.load_model('ckpt_04_tp008sl003_best_4h.hdf5')
df_minmax_1h= pd.read_csv('df_minmax_mrged_1h.csv')
df_minmax_4h= pd.read_csv('df_minmax_merged_4h.csv')
tesh_optimum_1h = pd.read_csv('df_best_final_EmbedRecolor_withoutResize_CorrectHL.csv')
tesh_optimum_4h = pd.read_csv('df_best_final_EmbedRecolor_withoutResize_4h.csv')
Selected_pairs_for_Old_model_4h= ['CHFJPY', 'AUDUSD', 'GBPCAD', 'EURJPY', 'GBPJPY', 'EURNZD', 'EURGBP',
                                  'NZDUSD', 'AUDCHF', 'AUDCAD', 'NZDCHF', 'EURCAD', 'EURUSD', 'GBPUSD', 'NZDCAD']
Selected_pairs_for_New_model_4h= ['USDCAD', 'EURAUD', 'EURCHF', 'GBPAUD', 'CADJPY', 'AUDNZD',
                                  'USDJPY', 'CADCHF', 'GBPCHF', 'AUDJPY', 'GBPNZD', 'NZDJPY', 'USDCHF']

KeyboardInterrupt: 

In [ ]:
mc = mpf.make_marketcolors(up='g',down='r')
s  = mpf.make_mpf_style(marketcolors=mc)

In [ ]:
def make_candle_stick(df, image_path):
    df.head()
    df['time']= df.time.apply(lambda x: dt.utcfromtimestamp(x).strftime('%Y%m%d %H%M%S'))
    df['time']= pd.to_datetime(df['time'])
    time_log= df.iloc[48]['time']
    
    openn = df.iloc[48]['open']
    high = df.iloc[48]['high']
    low = df.iloc[48]['low']
    close = df.iloc[48]['close']
    df = df.set_index('time')
    df.rename(columns={'tick_volume':'volume'}, inplace=True)
    
    HighLow= str(np.max(df.iloc[1:, 1])/np.min(df.iloc[1:, 2])).replace('.', '')[:6]
    if '_' in HighLow:
        HighLow = HighLow.replace('_', '0')
    
    #Heiken Ashi
    df_temp_Ha = df.copy()
    for i in range(1, len(df_temp_Ha)):
        df_temp_Ha.iloc[i, 0] = (df_temp_Ha.iloc[i-1, 0] + df_temp_Ha.iloc[i-1, 3])/2
        df_temp_Ha.iloc[i, 3] = (df_temp_Ha.iloc[i, 0] + df_temp_Ha.iloc[i, 1] + df_temp_Ha.iloc[i, 2] + df_temp_Ha.iloc[i, 3])/4
    
    time_log = str(time_log).replace(':', '').replace(' ', '').replace('-', '')
    
    img_name = "".join([pair, '_' ,str(time_log),'_', HighLow, '.jpg'])
    img_path= os.path.join(image_path, img_name)
    
    mplot(df_temp_Ha.iloc[1:-1, :], type='candle', volume=False, figscale=2,figratio=(10,10), style=s, savefig=img_path)
    return time_log, img_path, openn, high, low, close


In [ ]:
# font = ImageFont.truetype('times.ttf', 20)
font = cv.FONT_HERSHEY_COMPLEX
loc= (600, 795)
font_scale= 1.1
color = (0, 0, 0)
thickness= 3

# print(df_minmax)

In [ ]:
def makeEmbedImage(img_name, RecolPath, df_minmax):
    img = cv.imread(img_name)
    img = img[140:-207, 220: -120, :]
    hl = int(img_name.split('.')[0].split('_')[-1])
    if hl < 100:
        hl = hl * 10000
    elif hl < 1000:
        hl = hl * 1000
    elif hl < 10000:
        hl = hl * 100
    elif hl < 100000:
        hl = hl * 10

    mx = df_minmax.loc[0, pair]
    mn = df_minmax.loc[1, pair]
    hl = (hl - mn) / (mx - mn)
    w = np.uint8(((hl * 255)))
    img[771:850, 20:550, 2] = w
    img[771:850, 20:550, 0] = w
    img[771:850, 20:550, 1] = 0
    img = cv.putText(img, pair, loc, font, font_scale, color, thickness, cv.LINE_AA)
    img = cv.resize(img, (300, 300))
    name= img_name.split('\\')[-1]
    cv.imwrite(os.path.join(RecolPath, name), img)
    return img


In [ ]:
list_log = []
i = 0

# print(tesh_optimum.head())
while(True):
    df_log = pd.DataFrame()
    time = dt.now()
    min_part= str(time).split(':')[1]
    sec_part= str(time).split(':')[-1].split('.')[0]
    if (min_part == '59') & (int(sec_part) >= 48):
        dt_us_central = datetime.now(pytz.timezone('utc'))
        #print('GMT Time: ',dt_us_central.time())
        print(f'---- {time} ------ ')
        i+= 1
        for pair in currency_pairs:
#             tp= tesh_optimum.loc[tesh_optimum['p']== pair,'t'].values[0]
#             sl= tesh_optimum.loc[tesh_optimum['p']== pair,'s'].values[0]
#             acc_u= tesh_optimum.loc[tesh_optimum['p']== pair,'u'].values[0]
#             acc_d= tesh_optimum.loc[tesh_optimum['p']== pair,'d'].values[0]
            tp= 0.003
            sl= 0.004
            acc_u= 0.99
            acc_d= 0.01
            
            rates_1h = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_H1, 0, 50)
            df_1h=pd.DataFrame(rates_1h).reset_index(drop=True)
            time_log, img_name_1h, open_1h, high_1h, low_1h, close_1h = make_candle_stick(df_1h, image_path_1h)
            img_1h= makeEmbedImage(img_name_1h, RecolPath_1h, df_minmax_1h)
            acc_1h=model_1h.predict(np.array([img_1h,img_1h]))[0,1]
            
            rates_4h = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_H4, 0, 50)
            df_4h=pd.DataFrame(rates_4h).reset_index(drop=True)
            time_log, img_name_4h, open_4h, high_4h, low_4h, close_4h = make_candle_stick(df_4h, image_path_4h)
            img_4h= makeEmbedImage(img_name_4h, RecolPath_4h, df_minmax_4h)
            if pair in Selected_pairs_for_Old_model_4h:
                acc_4h=model_4h_old.predict(np.array([img_4h,img_4h]))[0,1]
            else:
                acc_4h=model_4h_new.predict(np.array([img_4h,img_4h]))[0,1]
            print(f'{pair}, acc_1h:{acc_1h}, acc_4h:{acc_4h}')
            
            if acc_1h >= acc_u:
                print('------ Buy Position -------')
                get_position_buy(pair, 0.1,sl , tp)
            elif acc_1h <= acc_d:
                print('--------- Sell Position --------')
                get_position_sell(pair, 0.1,sl , tp)
            
            list_log.append([time_log, pair, img_name_1h, img_name_4h, acc_1h, acc_4h,open_1h, high_1h, low_1h, close_1h, open_4h, high_4h, low_4h, close_4h])
            
            if pair == currency_pairs[-1]:
                arr_log = np.array(list_log)
                arr_log= arr_log.reshape(int(len(currency_pairs)*i), 14)
                df_temp = pd.DataFrame(arr_log, columns=['time_log','pair', 'img_name_1h','img_name_4h', 'acc_1h',
                                                         'acc_4h','open_1h', 'high_1h',  'low_1h','close_1h',
                                                         'open_4h','high_4h', 'low_4h', 'close_4h'])
                df_dashboard = df_temp[['pair', 'acc_1h', 'acc_4h']].rename(columns= {'pair': 'PAIR', 'acc_1h': '1H', 'acc_4h': '4H'})
                
                df_dashboard.to_exc('')
                if i == 1:
                    df_log = df_temp
                else:
                    df_log = pd.concat([df_log, df_temp], axis = 0)
                    
                file_name= r'C:\Users\Administrator\Desktop\Trade_1005\Logs_1005/df_log_{0}_3.csv'.format(str(i))
                df_log.to_csv(file_name)



In [32]:
crypto_list= ['BTCUSD', 'MKRUSD', 'XLMUSD', 'YFIUSD', 'DSHUSD', 'TRXUSD', 'CMPUSD', 'AVEUSD', 'ETHUSD', 'LNKUSD', 'UNIUSD', 
'BCHUSD', 'BATUSD', 'LTCUSD', 'EOSUSD', 'ADAUSD', 'MATUSD', 'ENJUSD']

In [33]:
for cypto in crypto_list:
    print(cypto)
    rates_1h = mt5.copy_rates_from_pos(cypto, mt5.TIMEFRAME_H1, 0, 50000)
    df_rates_1h= pd.DataFrame(rates_1h)
    if len(df_rates_1h)>0:
        df_rates_1h['time']= df_rates_1h.time.apply(lambda x: dt.utcfromtimestamp(x).strftime('%Y%m%d %H%M%S'))
        df_rates_1h['time']= pd.to_datetime(df_rates_1h['time'])
        df_rates_1h.iloc[:-1,:].to_csv(r'C:\Users\Administrator\Desktop\Crypto_Data\{0}_H1.csv'.format(cypto))
    
    rates_4h = mt5.copy_rates_from_pos(cypto, mt5.TIMEFRAME_H4, 0, 13000)
    df_rates_4h= pd.DataFrame(rates_4h)
    if len(df_rates_4h)>0:
        df_rates_4h['time']= df_rates_4h.time.apply(lambda x: dt.utcfromtimestamp(x).strftime('%Y%m%d %H%M%S'))
        df_rates_4h['time']= pd.to_datetime(df_rates_4h['time'])
        df_rates_4h.iloc[:-1,:].to_csv(r'C:\Users\Administrator\Desktop\Crypto_Data\{0}_H4.csv'.format(cypto))
                                        

BTCUSD
MKRUSD
XLMUSD
YFIUSD
DSHUSD
TRXUSD
CMPUSD
AVEUSD
ETHUSD
LNKUSD
UNIUSD
BCHUSD
BATUSD
LTCUSD
EOSUSD
ADAUSD
MATUSD
ENJUSD


In [27]:
rates_1h = mt5.copy_rates_from_pos('MKRUSD', mt5.TIMEFRAME_H1, 0, 5)
df_rates_1h= pd.DataFrame(rates_1h)
# df_rates_1h['time']= df_rates_1h.time.apply(lambda x: dt.utcfromtimestamp(x).strftime('%Y%m%d %H%M%S'))
# df_rates_1h['time']= pd.to_datetime(df_rates_1h['time'])
df_rates_1h

""


In [17]:
df_rates_1h= pd.DataFrame(rates_1h)
df_rates_1h

""


In [19]:
df_rates_1h['time']= df_rates_1h.time.apply(lambda x: dt.utcfromtimestamp(x).strftime('%Y%m%d %H%M%S'))
df_rates_1h['time']= pd.to_datetime(df_rates_1h['time'])
df_rates_1h

,time,open,high,low,close,tick_volume,spread,real_volume
0,2022-01-03 05:00:00,46998.57,47071.35,46725.35,46794.37,3961,1322,0
1,2022-01-03 06:00:00,46793.95,46937.79,46751.44,46825.81,3566,1329,0
2,2022-01-03 07:00:00,46825.24,47203.74,46784.98,47082.12,3579,1288,0
3,2022-01-03 08:00:00,47082.25,47100.69,46886.05,46925.27,3097,1294,0
4,2022-01-03 09:00:00,46925.33,46925.33,46856.96,46869.23,440,1406,0


In [12]:
len(crypto_list)

18